In [1]:
import os
from pathlib import Path
from operator import itemgetter
import pickle
import pydicom
from time import time
import pandas

import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection, Point, MultiPoint
from shapely.affinity import translate

from catch_converter.parse_contours import parse_cvi42ws
from LazyLuna.Mini_LL import *
from LazyLuna.CATCH_utils import *

In [2]:
# basepaths
bp       = '/Users/dietrichhadler/Desktop/Daten/CAMAID'
bp_ws    = '/Users/dietrichhadler/Desktop/Daten/CAMAID/Workspaces'
bp_annos = '/Users/dietrichhadler/Desktop/Daten/CAMAID/Gold'
bp_cases = '/Users/dietrichhadler/Desktop/Daten/CAMAID/Cases'
bp_imgs  = '/Users/dietrichhadler/Desktop/Daten/CAMAID/Imgs'

In [ ]:
# unpack the workspaces from ws_path to case_storage_path
#cvi42ws_paths     = [f for f in os.listdir(bp_ws) if f.endswith('cvi42ws')]
#parse_cvi42ws(bp_ws, bp_annos, process=True, debug=True)
#parse_cvi42ws(bp_ws, bp_annos, process=True, debug=False)

# get the paths of imgs and annotations via SeriesInstanceUIDs
imgsanno_paths = get_imgs_and_annotation_paths(bp_imgs, bp_annos)

print('Nr of path tuples: ', len(imgsanno_paths))
case_names = sorted([c[0] for c in imgsanno_paths])
from pprint import pprint
print('Existing Annotations: ', len(sorted([c[0] for c in imgsanno_paths if os.path.exists(c[1])])))
pprint(sorted([c[0] for c in imgsanno_paths if os.path.exists(c[1])]))

cases = []
sax_cine_view = SAX_CINE_View()
#sax_cs_view   = SAX_CS_View()
for count, (imgp,annop) in enumerate(imgsanno_paths):
    #if '008' not in imgp: continue
    print(count)
    print(os.path.basename(imgp), os.path.basename(annop))
    if not os.path.exists(imgp) or not os.path.exists(annop):
        print(os.path.exists(imgp), os.path.exists(annop))
        print('Not converting.'); print()
        continue
    st = time()
    try:
        case = Case(imgp, annop, os.path.basename(imgp), os.path.basename(bp_annos))
        case = sax_cine_view.customize_case(case)
        #case = sax_cs_view.customize_case(case)
        case.store(bp_cases)
        cases.append(case)
        print('Case customization took: ', time()-st, 'Case: ', case.reader_name, case.case_name)
    except Exception as e:
        print('Case customization failed: ', e)
    print()


In [ ]:
display(df['annotated'])
display(df.loc[df['annotated'].isin([0])])

In [ ]:
for k, v in df[['case','series_descr', 'annotated']].value_counts().iteritems():
    if k[-1]>0:
        print(k[-1], l)

In [ ]:
print('Cine SAX images:')
curr = None
for k,v in sorted(df[['case','series_descr']].value_counts().iteritems()):
    if curr is None or curr!=k[0]: print(k[0])
    curr = k[0]
    if 'RV 7 0' in k[1] or k[1]=='SAX CINE':
        print('\t', k[1], v)
print('Camaid_005_1 und Camaid_015_1 nicht segmentiert, da Vorhofflimmern')

In [ ]:
print('4CH View images:')

curr = None
for k,v in sorted(df[['case','series_descr']].value_counts().iteritems()):
    if curr is None or curr!=k[0]: print(k[0])
    curr = k[0]
    if 'cine_tf2d12' in k[1] and ('2 cv' in k[1] or '2cv' in k[1]):
        print('\t', k[1],v)
    if 'cine_tf2d12' in k[1] and ('3 cv' in k[1] or '3cv' in k[1]):
        print('\t', k[1], v)
    if 'cine_tf2d12' in k[1] and ('4 cv' in k[1] or '4cv' in k[1]):
        print('\t', k[1], v)
print('Several cases have two or more series of LAX')

In [ ]:
print('T1 images:')
for k,v in sorted(df[['case','series_descr']].value_counts().iteritems()):
    if curr is None or curr!=k[0]: print(k[0])
    curr = k[0]
    #if curr!='Camaid_011_2': continue
    if 'pre_MOLLI' in k[1] and 'T1' in k[1] and 'T1S' not in k[1] and '4cv' not in k[1]:
        print('\t', k[1], v)
    #print('\t', k[1], v)
print('T1 fehlt bei Camaid 11')

In [ ]:
print('T2 images:')
for k,v in sorted(df[['case','series_descr']].value_counts().iteritems()):
    if curr is None or curr!=k[0]: print(k[0])
    curr = k[0]
    #if curr!='Camaid_011_2': continue
    if 'T2' in k[1] and 'MOCO_T2' in k[1] and '4cv' not in k[1]:
        print('\t', k[1], v)
    #print('\t', k[1], v)
print('Viele Fälle bereits 3 aus 5 definiert.')

In [ ]:
def add_LL_tag(store_path, dcm, tag='Lazy Luna: None'): # Lazy Luna: SAX CS
    try:    dcm[0x0b, 0x10].value = tag
    except: dcm.private_block(0x000b, tag, create=True)
    dcm.save_as(filename=store_path, write_like_original=False)
    
all_cases = os.listdir(bp_imgs)
print('All cases: ', all_cases)

columns = ['case', 'series_descr', 'suid']
for i_c, c in enumerate(all_cases):
    for ip, p in enumerate(Path(os.path.join(bp_imgs, c)).glob('**/*.dcm')):
        p = str(p)
        dcm = pydicom.dcmread(p, stop_before_pixels=False)
        #print(dcm.SeriesDescription)
        try:
            if 'RV 7 0' in dcm.SeriesDescription: add_LL_tag(p, dcm, tag='Lazy Luna: SAX CINE')
            else:                                 add_LL_tag(p, dcm, tag='Lazy Luna: None')
        except:
            print('Failed at: Case', c)
            print(dcm)
            continue
    print(i_c, 'Done for', c)


In [ ]:
for i_c, c in enumerate(all_cases):
    for ip, p in enumerate(Path(os.path.join(bp_imgs, c)).glob('**/*.dcm')):
        p = str(p)
        dcm = pydicom.dcmread(p, stop_before_pixels=False)
        name = str(dcm[0x0b, 0x10].value).replace('Lazy Luna: ', '') # LL Tag
        print(name)

In [ ]:
# 1st: (tool 1)
# sort all images for one case
# get all annos for each image and offer that as info too
# add LL image tags for what we have here
#
# 2nd: (tool 2)
# make cases, AFTER images are defined
# created cases: connect annos to images
# 

In [3]:
case_path = '/Users/dietrichhadler/Desktop/Daten/CAMAID/Cases/Gold_Camaid_007_1_LL_case.pickle'
case = pickle.load(open(case_path, 'rb'))
print(case.case_name)

print(case.type)
print(case.categories)
print(case.other_categories)
print(case.all_imgs_sop2filepath.keys())
for k in case.all_imgs_sop2filepath.keys():
    print(k, len(case.all_imgs_sop2filepath[k]), end=', ')
for cr in case.crs:
    print(cr.name, cr.get_cr(), end=', ')

Camaid_007_1
SAX CINE
[]
{'SAX CINE': [<LazyLuna.Mini_LL.SAX_LV_ES_Category object at 0x12dc36610>, <LazyLuna.Mini_LL.SAX_LV_ED_Category object at 0x12dc36730>, <LazyLuna.Mini_LL.SAX_RV_ES_Category object at 0x12dc36850>, <LazyLuna.Mini_LL.SAX_RV_ED_Category object at 0x12dc36970>]}
dict_keys(['SAX CINE', 'SAX CS', 'SAX T1', 'SAX T2', 'LAX 2CV', 'LAX 3CV', 'LAX 4CV', 'SAX LGE', 'None'])
SAX CINE 540, SAX CS 0, SAX T1 0, SAX T2 0, LAX 2CV 0, LAX 3CV 0, LAX 4CV 0, SAX LGE 0, None 2756, LVESV 72.8618646822147, LVEDV 177.005327025968, RVESV 93.30839502980925, RVEDV 202.25544672603158, LVSV 104.1434623437533, LVEF 58.836343568849934, RVSV 108.94705169622233, RVEF 53.866065641366724, LVMYOMASS 100.65314827153077, RVMYOMASS 0.0, LVESP 13, RVESP 13, LVEDP 0, RVEDP 0, LVEDP 18, 

In [4]:
view = SAX_CINE_View()
case = view.customize_case(case)
print(case.case_name)

print(case.type)
print(case.categories)
print(case.other_categories)
print(case.all_imgs_sop2filepath.keys())
for k in case.all_imgs_sop2filepath.keys():
    print(k, len(case.all_imgs_sop2filepath[k]), end=', ')
for cr in case.crs:
    print(cr.name, cr.get_cr(), end=', ')

Camaid_007_1
SAX CINE
[<LazyLuna.Mini_LL.SAX_LV_ES_Category object at 0x12dc36610>, <LazyLuna.Mini_LL.SAX_LV_ED_Category object at 0x12dc36730>, <LazyLuna.Mini_LL.SAX_RV_ES_Category object at 0x12dc36850>, <LazyLuna.Mini_LL.SAX_RV_ED_Category object at 0x12dc36970>]
{'SAX CINE': [<LazyLuna.Mini_LL.SAX_LV_ES_Category object at 0x12dc36610>, <LazyLuna.Mini_LL.SAX_LV_ED_Category object at 0x12dc36730>, <LazyLuna.Mini_LL.SAX_RV_ES_Category object at 0x12dc36850>, <LazyLuna.Mini_LL.SAX_RV_ED_Category object at 0x12dc36970>]}
dict_keys(['SAX CINE', 'SAX CS', 'SAX T1', 'SAX T2', 'LAX 2CV', 'LAX 3CV', 'LAX 4CV', 'SAX LGE', 'None'])
SAX CINE 540, SAX CS 0, SAX T1 0, SAX T2 0, LAX 2CV 0, LAX 3CV 0, LAX 4CV 0, SAX LGE 0, None 2756, LVESV 72.8618646822147, LVEDV 177.005327025968, RVESV 93.30839502980925, RVEDV 202.25544672603158, LVSV 104.1434623437533, LVEF 58.836343568849934, RVSV 108.94705169622233, RVEF 53.866065641366724, LVMYOMASS 100.65314827153077, RVMYOMASS 0.0, LVESP 13, RVESP 13, LVEDP 